In [15]:
!pip install sagemaker --upgrade
!pip install boto3 --upgrade
!pip install botocore --upgrade


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [123]:
from sagemaker.serializers import CSVSerializer, JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.explainer import (
    ExplainerConfig, 
    ClarifyExplainerConfig, 
    ClarifyShapConfig, 
    ClarifyInferenceConfig, 
    ClarifyShapBaselineConfig, 
    ClarifyTextConfig
)

json_serializer = JSONSerializer()
json_deserializer = JSONDeserializer()
csv_serializer = CSVSerializer()
shap_baseline = csv_serializer.serialize([["<UNK>"]])

explainer_config_dict = {
    'ClarifyExplainerConfig': {
        'InferenceConfig': {
            'FeaturesAttribute': '[].features',
            'ContentTemplate': '$records',
            'RecordTemplate': '{"features":$features}',
            'LabelAttribute': '[].predictions[]',
            'ProbabilityAttribute': '[].probabilities[]',
            'FeatureTypes': ['text'],
            'FeatureHeaders': ['ItemDescription'],
        },
        'ShapConfig': {
            'ShapBaselineConfig': {
                'MimeType': 'text/csv',
                'ShapBaseline': '<UNK>'
            },
            'NumberOfSamples': 5,
            'TextConfig': {
                'Language': 'en', 
                'Granularity': 'sentence'
            }
        }
    }
}
explainer_config_dict

{'ClarifyExplainerConfig': {'InferenceConfig': {'FeaturesAttribute': '[].features',
   'ContentTemplate': '$records',
   'RecordTemplate': '{"features":$features}',
   'LabelAttribute': '[].predictions[]',
   'ProbabilityAttribute': '[].probabilities[]',
   'FeatureTypes': ['text'],
   'FeatureHeaders': ['ItemDescription']},
  'ShapConfig': {'ShapBaselineConfig': {'MimeType': 'text/csv',
    'ShapBaseline': '<UNK>'},
   'NumberOfSamples': 5,
   'TextConfig': {'Language': 'en', 'Granularity': 'sentence'}}}}

In [127]:
import boto3
boto3_session = boto3.session.Session()
sagemaker_client = boto3.client("sagemaker")

sagemaker_runtime_client = boto3.client("sagemaker-runtime")
sagemaker_session = sagemaker.Session(
    boto_session=boto3_session,
    sagemaker_client=sagemaker_client,
    sagemaker_runtime_client=sagemaker_runtime_client,
)

version = "application-json-multiple-items"
endpoint_name = f'nlp-ole-example-with-clarify-{version}'
sagemaker_client.delete_model(ModelName=endpoint_name)    
sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)



{'ResponseMetadata': {'RequestId': '32c561fa-b6d2-4339-bf21-5a1a06928d67',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '32c561fa-b6d2-4339-bf21-5a1a06928d67',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 16 Jan 2024 22:48:48 GMT'},
  'RetryAttempts': 0}}

In [128]:
from datetime import datetime
import time

import sagemaker
from sagemaker.model import Model
from sagemaker import get_execution_role


print("Current Date and Time:", datetime.now())


role = get_execution_role()
image_uri = f'123456789012.dkr.ecr.us-west-2.amazonaws.com/sagemaker-studio-d-ebafjgrwlbqa:{version}'

start = time.time()

response = sagemaker_client.create_model(
    ModelName = endpoint_name,
    ExecutionRoleArn = role,
    Containers = [{
        "Image": image_uri,
        "Mode": "SingleModel",
    }]
)

sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_name,
    ProductionVariants=[
        {
            "VariantName": "TestVariantName",
            "ModelName": endpoint_name,
            "InitialInstanceCount": 1,
            "InstanceType": 'ml.m4.xlarge',
        }
    ],
    ExplainerConfig=explainer_config_dict,
)

response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_name,
)

sagemaker_session.wait_for_endpoint(endpoint_name)

print("Deployed after mins:", int(time.time() - start) / 60.0)
response

Current Date and Time: 2024-01-16 22:48:54.517058
----!Deployed after mins: 2.533333333333333


{'EndpointArn': 'arn:aws:sagemaker:us-west-2:123456789012:endpoint/nlp-ole-example-with-clarify-application-json-multiple-items',
 'ResponseMetadata': {'RequestId': 'aef8d4e6-4db9-44ff-9824-4f7d7f283860',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'aef8d4e6-4db9-44ff-9824-4f7d7f283860',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '128',
   'date': 'Tue, 16 Jan 2024 22:48:55 GMT'},
  'RetryAttempts': 0}}

In [129]:
sagemaker_client.describe_endpoint(EndpointName=endpoint_name)["ExplainerConfig"]

{'ClarifyExplainerConfig': {'InferenceConfig': {'FeaturesAttribute': '[].features',
   'ContentTemplate': '$records',
   'RecordTemplate': '{"features":$features}',
   'ProbabilityAttribute': '[].probabilities[]',
   'LabelAttribute': '[].predictions[]',
   'FeatureHeaders': ['ItemDescription'],
   'FeatureTypes': ['text']},
  'ShapConfig': {'ShapBaselineConfig': {'MimeType': 'text/csv',
    'ShapBaseline': '<UNK>'},
   'NumberOfSamples': 5,
   'TextConfig': {'Language': 'en', 'Granularity': 'sentence'}}}}

In [134]:
import json
import boto3
import pprint
boto3_session = boto3.session.Session()
sagemaker_runtime_client = boto3.client("sagemaker-runtime")

data = json_serializer.serialize([{"features": ["hello"], "features":["value"]}])

print(f"Calling: {endpoint_name}")

response = sagemaker_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    Body=data,
    # EnableExplanations="`false`", # "`false`" or "`true`"
)

result = json_deserializer.deserialize(response["Body"], content_type=response["ContentType"])
pprint.pprint(result)

Calling: nlp-ole-example-with-clarify-application-json-multiple-items
{'explanations': {'kernel_shap': [[{'attributions': [{'attribution': [0.44721362665340225],
                                                      'description': {'partial_text': 'hello',
                                                                      'start_idx': 0}}],
                                    'feature_header': 'ItemDescription',
                                    'feature_type': 'text'}]]},
 'label_headers': ['auto-generated-description-for-hello'],
 'predictions': {'content_type': 'application/json',
                 'data': '[{"predictions": '
                         '"auto-generated-description-for-hello", '
                         '"probabilities": 0.6614720723421877}]'},
 'version': '1.0'}
